# Face Morphing

## Prepare Environment

In [ ]:
#@title Download Code { display-mode: "form" } 
import os
os.chdir('/content')
CODE_DIR = 'face-editing'

## clone repo
!git clone https://github.com/estephe-arnaud/face-editing.git $CODE_DIR

## install ninja
!wget https://github.com/ninja-build/ninja/releases/download/v1.8.2/ninja-linux.zip
!sudo unzip ninja-linux.zip -d /usr/local/bin/
!sudo update-alternatives --install /usr/bin/ninja ninja /usr/local/bin/ninja 1 --force

## change work directory
os.chdir(f'./{CODE_DIR}')

In [ ]:
#@title Download Models { display-mode: "form" } 
!wget "https://nextcloud.univ-lille.fr/index.php/s/8RrH7MzLGFfwMjK/download/pretrained_models.tar.xz" && tar -xvJf ./pretrained_models.tar.xz && rm ./pretrained_models.tar.xz

In [ ]:
#@title Load Face Model { display-mode: "form" } 
from models.face_model import FaceModel
FACE_MODEL = FaceModel()

In [ ]:
#@title Import Modules { display-mode: "form" } 
import os
from datetime import datetime
from numpy import linspace
from PIL import Image
from IPython.core.display import Video

from utils.common import *
from utils.data_utils import *
from utils.inference_utils import *
from utils.morphing_utils import *

In [ ]:
#@title Morphing Functions { display-mode: "form" }
def morphing(A, B, n_frames):
    images = []    
    ratios = linspace(0, 1, n_frames).tolist()

    latent_A, weights_deltas_A = A["latent"], A["weights_deltas"]
    latent_B, weights_deltas_B = B["latent"], B["weights_deltas"]
    
    for ratio in ratios:
        latent = interpolate_latents(latent_A, latent_B, ratio)
        weights_deltas = interpolate_weights_deltas(weights_deltas_A, weights_deltas_B, ratio)

        x = FACE_MODEL.decoder(latent=latent, weights_deltas=weights_deltas, resize=False)
        image = tensor2im(x[0])
        # image = getMesh(image)
        images.append(image)
        
    return images

def run(input_dir, output_dir, n_frames):
    inputs = [Image.open(f) for f in image_files(input_dir)]
    inputs = [exif_transpose(image) for image in inputs]
    inputs.append(inputs[0])
    
    data = run_prediction(FACE_MODEL, inputs, fine_encoding=False, return_weights_deltas=True)    

    pts_A = [{"latent": data["latent"][i], "weights_deltas": data["weights_deltas"][i]} for i in range(0, len(inputs))]
    pts_B = [{"latent": data["latent"][i], "weights_deltas": data["weights_deltas"][i]} for i in range(1, len(inputs))]

    outputs = []
    for p_A, p_B in zip(pts_A, pts_B):
        outputs += morphing(p_A, p_B, n_frames)

    # Save video
    os.makedirs(output_dir, exist_ok=True)
    filename = datetime.now().strftime("%Y%m%d-%H%M%S")
    save_path = "{}/{}.mp4".format(output_dir, filename)
    create_video(outputs, save_path, mode="cv2")
    Video(save_path)

## Perform Inference

In [ ]:
#@title Define Inference Parameters { display-mode: "form" } 
input_dir = "/content" #@param {type:"string"}
output_dir = "/content" #@param {type:"string"}
n_frames = 50 #@param {type:"slider", min:1, max:100, step:5}

run(input_dir, output_dir, n_frames)